In [1]:
import ROOT
from fitter import FitHandler
import numpy as np

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8b7ba20


In [2]:
default_lumi = 2000
default_hel = {
    "lumi_frac": 1.,
    "e_pol": 0.,
    "p_pol": 0.,
}

# https://arxiv.org/pdf/1506.07830
# However, https://arxiv.org/pdf/2503.19983 cites the above but assigns 0.45 to both mixed pols
# LCVision scenario uses 3 ab_inv instead of 2
ilc_250_h20_lumi = 2000
ilc_250_h20_hel = [
    {
        "lumi_frac": 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi_frac": 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi_frac": 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]

# run = ilc_250_h20[0]

In [3]:
conf = {
    "parameters": {
        "g1z": 0.0,
        "ka": 0.0,
        "la": 0.0,
    },
    "obs_names": [
        "O_g1z_pos_1em05",
        "O_ka_pos_1em05",
        "O_la_pos_1em05",
    ],
    "signal_cat": "4f_sw_sl_signal",
    "signal_processes": [
        "4f_sw_sl_eLpL_signal",
        "4f_sw_sl_eLpR_signal",
        "4f_sw_sl_eRpL_signal",
        "4f_sw_sl_eRpR_signal",
    ],
    "backgrounds": [
        "4f_sl_bkg",
        "4f_not_sl",
        "2f",
        "3f",
        "5f",
        "6f",
        "higgs",
    ],
    "n_bins": 65,
}
input_path = "data/histograms/full/raw_histograms.root"

In [4]:
fit_handler = FitHandler(input_path, conf)

In [5]:
# ws = fit_handler.build_model([run])
run_conf = (default_lumi, [default_hel])
# run_conf = (ilc_250_h20_lumi, ilc_250_h20_hel)
ws = fit_handler.build_model(*run_conf)
# coupling_pars = fit_handler.coupling_pars
coupling_pars = [ws.var(name) for name in conf["parameters"]]
obs_pars = [ws.var(f"{name}_0") for name in conf["obs_names"]]
# obs_pars = [ws.var(f"{name}_{idx}") for idx in range(len(run_conf[1])) for name in conf["obs_names"] ]

{'lumi_frac': 1.0, 'e_pol': 0.0, 'p_pol': 0.0}
hello
start constructor
finished constructor
start constructor
finished constructor
start constructor
finished constructor
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooMultiVarGaussian::multi_gauss_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_g1z_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_ka_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::O_la_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooFunctorBinding::expectation_O_g1z_pos_1em05_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooProduct::lumi_0
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooConstVar::1
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::total_lumi
[#1] INFO:ObjectHandling -- RooWorkspace::import(ws) importing RooRealVar::e_pol_0
[#1] INFO:Obje

In [6]:
ws.Print("t")


RooWorkspace(ws) ws contents

variables
---------
(O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0,e_pol_0,g1z,ka,la,mu_signal,p_pol_0,total_lumi)

p.d.f.s
-------
RooMultiVarGaussian::multi_gauss_0[ x=(O_g1z_pos_1em05_0,O_ka_pos_1em05_0,O_la_pos_1em05_0) mu=(expectation_O_g1z_pos_1em05_0,expectation_O_ka_pos_1em05_0,expectation_O_la_pos_1em05_0) ] = 1
  RooFunctorBinding::expectation_O_g1z_pos_1em05_0[ function=0x2175da00 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -131325
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000
  RooFunctorBinding::expectation_O_ka_pos_1em05_0[ function=0x9e9b780 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -326754
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000
  RooFunctorBinding::expectation_O_la_pos_1em05_0[ function=0x22997a70 vars=(lumi_0,e_pol_0,p_pol_0,g1z,ka,la,mu_signal) ] = -161778
    RooProduct::lumi_0[ 1 * total_lumi ] = 2000



In [7]:
model = ws.pdf("multi_gauss_0")
# model = ws.pdf("sim_pdf")

In [8]:
toy_data = fit_handler.make_toy_obs(default_lumi, default_hel)
print(toy_data)

[-129720.12421827164, -326892.7154365902, -162723.25256563287]


In [9]:
# n = 1000000
n = 500000
model_ds = model.generate(obs_pars, NumEvents=n)
model_ds.Print("v")

DataStore multi_gauss_0Data (Generated From multi_gauss)
  Contains 500000 entries
  Observables: 
    1)  O_g1z_pos_1em05_0 = -129268  L(-135092 - -127558)  "O_g1z_pos_1em05_0"
    2)   O_ka_pos_1em05_0 = -326683  L(-330045 - -323462)  "O_ka_pos_1em05_0"
    3)   O_la_pos_1em05_0 = -161280  L(-166859 - -156697)  "O_la_pos_1em05_0"


In [10]:
cov_matrix = model_ds.covarianceMatrix()


In [11]:

cov_matrix_np = np.ndarray((3, 3), dtype=np.double, buffer=cov_matrix.GetMatrixArray())
print(cov_matrix_np)

[[1522374.11575699  344787.0184021  -732699.85147095]
 [ 344787.0184021  1171459.94151306  190230.94810486]
 [-732699.85147095  190230.94810486 2798138.35060883]]


In [12]:
np.sqrt(np.linalg.inv(cov_matrix_np))

/tmp/lreichen/ipykernel_25295/1425982355.py:1: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(np.linalg.inv(cov_matrix_np))


array([[0.00091435,        nan, 0.00048814],
       [       nan, 0.00097988,        nan],
       [0.00048814,        nan, 0.0006552 ]])

In [13]:
frame0 = obs_pars[0].frame()
frame1 = obs_pars[1].frame()
frame2 = obs_pars[2].frame()
model_ds.plotOn(frame0)
model_ds.plotOn(frame1)
model_ds.plotOn(frame2)

In [14]:
c0 = ROOT.TCanvas()
frame0.Draw()
c0.Draw()

c1 = ROOT.TCanvas()
frame1.Draw()
c1.Draw()

c2 = ROOT.TCanvas()
frame2.Draw()
c2.Draw()

In [15]:
# now I need to make some toys with my own code...
n_small = 200
my_ds = []
for i in range(n_small):
    my_ds.append(fit_handler.make_toy_obs(default_lumi, default_hel, seed=i))

In [16]:
my_ds_np = np.asarray(my_ds)

In [17]:
my_ds_cov = np.cov(my_ds_np.T)
print(my_ds_cov)

[[1196885.95765315  224391.91655165 -419619.4268849 ]
 [ 224391.91655165 1058201.80219543  398915.48285186]
 [-419619.4268849   398915.48285186 2898164.91601036]]


In [18]:
print(cov_matrix_np / my_ds_cov)

[[1.27194584 1.53653939 1.74610565]
 [1.53653939 1.10702887 0.47687031]
 [1.74610565 0.47687031 0.96548624]]
